In [1]:
# All functions are defined here. Assumed access to global variable.
#############################################################################################################################################################################

def Generate_B(Type: str, n: str, c_list):      # Returns the initial exchange matrix B from a coxeter element c
    W1 = WeylGroup(Type+str(n))
    Car = CartanMatrix(W1)
    B = copy(Car)
    W2 = WeylGroup(B)
    c1 = W2.from_reduced_word(c_list)
    allReducedWords = c1.reduced_words()

    for i in range(n):
        for j in range(n):
            count = 0
            if i==j:
                B[i,j] = 0
            if B[i,j] == 0:
                continue
            for k in allReducedWords:
                if c_list.index(i)<c_list.index(j):
                    count = count + 1
                elif count > 0:
                    B[i,j] = 0
                    break
            if count == len(allReducedWords):
                B[i,j]=-B[i,j]
    return(B)
##############################################################################################################################################################################

def Cartan_Counterpart(B):                      # Returns the Cartan counterpart of a matrix B
    a, b = B.dimensions()
    C = zero_matrix(QQ, b)
    for i in range(a):
        for j in range(b):
            C[i, j] = -abs(B[i, j])
    C = C + 2*identity_matrix(QQ, b)
    return(C) 
##############################################################################################################################################################################

def Weyl_action(L, WElt):                       # Returns the image of L acted by a Weyl group element WElt (WElt as a list)
    m = len(WElt)
    for i in range(m):
        index = WElt[m-i-1]
        L = Simple_Reflections[index](L)
    return(L)
##############################################################################################################################################################################

def Tau_c(L):                                   # Returns the image of L under Tau_c        # Input: g-vectors in weight space      # Output: g-vectors in weight space
    for i in Lambdas:
        if L == -i:
            return(i)
    return(Weyl_action(L, c_List))
##############################################################################################################################################################################

def Tau_c_inverse(L):                           # Returns the image of L under Tau_c inverse        # Input: g-vectors in weight space      # Output: g-vectors in weight space
    for i in Lambdas:
        if L == i:
            return(-i)
    return(Weyl_action(L, c_Inverse_List))
##############################################################################################################################################################################

def Comp_Deg(L1, L2):                           # Returns the compatibility degree between g-vectors L1 and L2      # Input: g-vectors in weight space      # Output: int
    count = 0
    while True:
        for i in range(n):
            if L1 == Lambdas[i]:
                count = 1
                Index = i
                break
        if count == 1:
            L = Weyl_action(L2, c_Inverse_List)-L2
            X = vector([L[i] for i in range(n)])                    # Coeffs of in fundamental weights
            Y = Lambda_to_Alpha*X                                   # inverse: each col is lambda in terms of alpha, so Y is L in terms of alpha
            answer = max(Y[Index],0)                                # [L: alpha_i]_+
            count = 0
            break
        L1 = Tau_c(L1)
        L2 = Tau_c(L2)
    return(answer)
##############################################################################################################################################################################

def CompDegMatrix():                            # Returns a matrix whose entry (i,j) is (x_j||x_i), elements ordered as in gVector_List
    CDM = zero_matrix(QQ, Number_of_Cluster_Variables)
    for i in range(Number_of_Cluster_Variables):
        for j in range(Number_of_Cluster_Variables):
            CDM[j,i] = Comp_Deg(gVector_as_Weights_List[i],gVector_as_Weights_List[j])         # Column i is comp deg (xi || __)
    return(gVector_List, CDM)
##############################################################################################################################################################################

def Generate_Exchangeable_Pairs():              # Returns the List of indices of pairs of exchangeable variables
    Pairs = []
    for i in range(Number_of_Cluster_Variables):
        for j in range(i,Number_of_Cluster_Variables):
            if CDMatrix[i,j] == 1 and CDMatrix[j,i] == 1:
                Pairs.append([i,j])
    return(Pairs)
##############################################################################################################################################################################   

def Cluster_Monomial_RHS(Index1, Index2):       # Returns the RHS of the relation in the exchange between Index1 and Index2
                                                # Output is 2 lists: a list of index indicating a compatible set, and a list of degree vectors
    Comp_with_Index1=set()
    Comp_with_Index2=set()
    for i in range(Number_of_Cluster_Variables):
        if CDMatrix[Index1, i] == 0:                        # x_i is compatible with the var corresp to Beta1
            Comp_with_Index1.add(i)
        if CDMatrix[Index2, i] == 0:
            Comp_with_Index2.add(i)
    Comp_with_Both = Comp_with_Index1.intersection(Comp_with_Index2)
    Comp_with_Both_List = list(Comp_with_Both)                                 # convert to list for the for loop below
    k=len(Comp_with_Both_List)

    Possible_CDSubMatrix_modified = CDMatrix[Comp_with_Both_List, Comp_with_Both_List] - identity_matrix(ZZ, k)
    Matrix_for_Graph = matrix(ZZ, k)
    for i in range(k):
        for j in range(k):
            Matrix_for_Graph[i,j] = (Possible_CDSubMatrix_modified[i,j] == 0)

    Possible_Graph = Graph(Matrix_for_Graph, format='adjacency_matrix')
    tmp = [set(i) for i in Possible_Graph.cliques_maximum()]
    Possible_Seed_Index = set.intersection(*tmp)                    # this is the set of vertices contained in all complete subgraphs

    Res_Possibles = [Comp_with_Both_List[i] for i in Possible_Seed_Index]
    Num_Possible = len(Res_Possibles)

    count = 0
    Coeff_List=[]
    while len(Coeff_List)<2:
        Res_Possibles_Vectors = copy([gVector_as_Weights_List[i] for i in Res_Possibles])
        tmp1 = gVector_as_Weights_List[Index1]
        tmp2 = gVector_as_Weights_List[Index2]
        for i in range(count):
            tmp1 = Tau_c(tmp1)
            tmp2 = Tau_c(tmp2)
            for j in range(Num_Possible):
                Res_Possibles_Vectors[j] = Tau_c(Res_Possibles_Vectors[j])
        #Solve c_i by matrix equation
        X = matrix([v[j] for j in range(n)]for v in Res_Possibles_Vectors)
        tmpsum = tmp1 + tmp2
        b = vector(tmpsum[j] for j in range(n))
        coeff = X.solve_left(b)
        if coeff not in Coeff_List:
            Coeff_List.append(coeff)
        count = count + 1
        
    Coeff_List.sort(key = str)
    return(Res_Possibles, Coeff_List)
##############################################################################################################################################################################
    
def Generate_Exchange_Relations():              # Returns 4 lists:  # List of exchangeable pairs, List of cluster variables on RHS of exchange relation, 
                                                                    # List of degree vectors of 1st monomial, List of degree vectors of 2nd monomial  
    Crossing_List = Generate_Exchangeable_Pairs()
    Cluster_Index_List = []
    Coeff_1_List = []
    Coeff_2_List = []
    for i in Crossing_List:
        Index1 = i[0]
        Index2 = i[1]
        [Cluster_List, Coefficients] = Cluster_Monomial_RHS(Index1, Index2)
        Cluster_Index_List.append(Cluster_List)
        Coeff_1_List.append(Coefficients[0])
        Coeff_2_List.append(Coefficients[1])
    return(Crossing_List, Cluster_Index_List, Coeff_1_List, Coeff_2_List)
##############################################################################################################################################################################   

def Weighted_Degree():                          # Returns 3 matrices whose entries are weighted degrees of each monomial in each exchange relation, cols ordered as in Crossing_List
    [Crossing_List, Cluster_Index_List, Coeff_1_List, Coeff_2_List] = Generate_Exchange_Relations()
    Num_of_Exch_Rel = len(Crossing_List)
    Weighted_Deg_of_Crossing_Matrix = matrix(Number_of_Cluster_Variables, Num_of_Exch_Rel)
    Weighted_Deg_of_Monomial_1_Matrix = matrix(Number_of_Cluster_Variables, Num_of_Exch_Rel)
    Weighted_Deg_of_Monomial_2_Matrix= matrix(Number_of_Cluster_Variables, Num_of_Exch_Rel)
    for i in range(Number_of_Cluster_Variables):
        for j in range(Num_of_Exch_Rel):
            Index1 = Crossing_List[j][0]
            Index2 = Crossing_List[j][1]
            Weighted_Deg_of_Crossing_Matrix[i,j] = CDMatrix[Index1, i] + CDMatrix[Index2, i]
            Cluster_Index = Cluster_Index_List[j]
            Coeff_1 = Coeff_1_List[j]
            Coeff_2 = Coeff_2_List[j]
            Cluster_Size = len(Cluster_Index)
            Weight_Deg_1 = 0
            Weight_Deg_2 = 0
            for k in range(Cluster_Size):
                Weight_Deg_1 = Weight_Deg_1 + Coeff_1[k] * CDMatrix[Cluster_Index[k],i]
                Weight_Deg_2 = Weight_Deg_2 + Coeff_2[k] * CDMatrix[Cluster_Index[k],i]
            Weighted_Deg_of_Monomial_1_Matrix[i,j] = Weight_Deg_1
            Weighted_Deg_of_Monomial_2_Matrix[i,j] = Weight_Deg_2
    return(Weighted_Deg_of_Crossing_Matrix, Weighted_Deg_of_Monomial_1_Matrix, Weighted_Deg_of_Monomial_2_Matrix, Num_of_Exch_Rel, Crossing_List)
##############################################################################################################################################################################

def Compare_Weighted_Degree():                  # Returns a matrix and a boolean value
                                                # CompareMatrix: entries are weighted degree of crossing term - max weighted degree of RHS
                                                # Test_For_Whole_Matrix: checks the only nonzero entries in each column correspond to a pair of exchanging variables
    [Weighted_Deg_of_Crossing_Matrix, Weighted_Deg_of_Monomial_1_Matrix, Weighted_Deg_of_Monomial_2_Matrix, Num_of_Exch_Rel, Crossing_List] = Weighted_Degree()
    Max_Weighted_Deg_of_RHS = matrix(Number_of_Cluster_Variables, Num_of_Exch_Rel)
    CompareMatrix = matrix(Number_of_Cluster_Variables, Num_of_Exch_Rel)
    for i in range(Number_of_Cluster_Variables):
        for j in range(Num_of_Exch_Rel):
            Max_Weighted_Deg_of_RHS[i,j] = max(Weighted_Deg_of_Monomial_1_Matrix[i,j],Weighted_Deg_of_Monomial_2_Matrix[i,j])
            CompareMatrix[i,j] = Weighted_Deg_of_Crossing_Matrix[i,j] - Max_Weighted_Deg_of_RHS[i,j]
    Test_For_Whole_Matrix = True
    for j in range(Num_of_Exch_Rel):
        Column_j = CompareMatrix[:,j]
        Nonzero_Indices = []
        for i in range(Number_of_Cluster_Variables):
            if Column_j[i] != 0:
                Nonzero_Indices.append(i)
        if Nonzero_Indices != Crossing_List[j]:
            Test_For_Whole_Matrix = False
    return(CompareMatrix, Test_For_Whole_Matrix)
##############################################################################################################################################################################

def Check_Orbit_Size_of_T():                    # Returns the sizes of orbits under T action (output as a set, so no repeating element)
    OrbitSize = list()
    for i in gVector_as_Weights_List:
        tmp = Tau_c(i)
        count = 1
        while tmp != i:
            tmp = Tau_c(tmp)
            count = count + 1
        OrbitSize.append(count)
    return(set(OrbitSize))
##############################################################################################################################################################################   


In [ ]:
# All global variables and main block are here.
                                                                # These two initialize everything. Change them to change the type being computed.
Type = 'F'
n = 4

#############################################################################################################################################################################

c_List = list(range(n))
B_Exchange_Matrix = Generate_B(Type, n, c_List)
A_Cluster_Algebra = ClusterAlgebra(B_Exchange_Matrix)                           # The cluster algebra generated by exchange matrix B
A_Cluster_Algebra.explore_to_depth(infinity)
gVector_List = A_Cluster_Algebra.g_vectors_so_far()                             # The list of all g-vectors (list of tuples)
C_CartanMatrix = Cartan_Counterpart(B_Exchange_Matrix)                          # The Cartan Counterpart of exchange matrix B
W_WeylGroup = WeylGroup(C_CartanMatrix)                                         # Weyl Group defined by a Cartan matrix (type: matrix)
R_RootSystem = RootSystem(W_WeylGroup)
c_CoxeterElement = W_WeylGroup.from_reduced_word(c_List)                        # Chosen Coxeter element c by c_list
c_Inverse_List = (c_CoxeterElement^-1).reduced_word()    

Number_of_Cluster_Variables = len(gVector_List)                                 # Number of all cluster variables 

Weight_Space = R_RootSystem.weight_space()
Alphas = Weight_Space.simple_roots()                                            # Simple roots in weight space, in terms of fundamental weights
Lambdas = Weight_Space.fundamental_weights()                                    # Fundamental weights
Simple_Reflections = Weight_Space.simple_reflections()                          # Simple reflections

Alpha_to_Lambda = transpose(Matrix([vector(Alphas[i]) for i in range(n)]))      # Each col is alpha in terms of lambda
Lambda_to_Alpha = Alpha_to_Lambda.inverse()                                     # Each col is lambda in terms of alpha

gVector_as_Weights_List = []                                                    # List of all g-vectors in weight space, same order as in gVector_List
for i in range(Number_of_Cluster_Variables):
    GWElt = 0
    for j in range(n):
        GWElt = GWElt + gVector_List[i][j]*Lambdas[j]
    gVector_as_Weights_List.append(GWElt)

CDMatrix = CompDegMatrix()[1]
[Weighted_Deg_of_Crossing_Matrix, Weighted_Deg_of_Monomial_1_Matrix, Weighted_Deg_of_Monomial_2_Matrix, Num_of_Exch_Rel, Crossing_List] = Weighted_Degree()
[CompareMatrix, Test_For_Whole_Matrix] = Compare_Weighted_Degree()
Orbit_Sizes = Check_Orbit_Size_of_T()

print('Type:',Type+str(n),'\n')
print('gVector_List','\n'+str(matrix(gVector_List)),'\n')
print('CDMatrix','\n'+str(CDMatrix),'\n')
print('Crossing_List','\n'+str(matrix(Crossing_List)),'\n')
print('CompareMatrix','\n'+str(matrix(CompareMatrix)),'\n')
print('Test_For_Whole_Matrix','\n'+str(Test_For_Whole_Matrix),'\n')
print('Orbit_Sizes','\n'+str(Orbit_Sizes),'\n')

